In [5]:
import numpy as np
import time
import scipy.io as sio
import torch
import augment
import torchaudio
import python_speech_features as psf
import random
from os import path
import glob
from scipy.io import wavfile
import scipy
import soundfile as sf
import librosa
import os

In [12]:
from tqdm import tqdm 

In [2]:
! python -m pip install git+https://github.com/facebookresearch/WavAugment.git

  Cloning https://github.com/facebookresearch/WavAugment.git to /tmp/pip-req-build-iekv9w6d
  Created wheel for augment: filename=augment-0.2-py3-none-any.whl size=5372 sha256=ca497affcfe62875d4242ed900c9445708b2f6eca3787e2822cf1656057f6305
  Stored in directory: /tmp/pip-ephem-wheel-cache-e4p5mrko/wheels/de/31/23/bb3c93de3e7e94bda500cba8eff0451435374f4e713d22b0a1
Successfully built augment
You should consider upgrading via the '/home/akansh/anaconda3/bin/python -m pip install --upgrade pip' command.


In [4]:
!pip install python_speech_features

  Created wheel for python-speech-features: filename=python_speech_features-0.6-py3-none-any.whl size=5889 sha256=955b4ae664dc2969f4d72a4bb295d11f69b1642de54076c8af2b927bbdd3ef8b
  Stored in directory: /home/akansh/.cache/pip/wheels/b0/0e/94/28cd6afa3cd5998a63eef99fe31777acd7d758f59cf24839eb
Successfully built python-speech-features
You should consider upgrading via the '/home/akansh/anaconda3/bin/python -m pip install --upgrade pip' command.


In [ ]:
augment_type=['reverb','uniform_noise']
loc='/data1/shivani/';
data_path=loc+'data_clipped/';
dest_loc1=loc+'data_clipped_'+augment_type[0] +'/';
dest_loc2=loc+'data_clipped_'+augment_type[1] +'/';

if not os.path.isdir(dest_loc1):
    os.mkdir(dest_loc1);
if not os.path.isdir(dest_loc2):
    os.mkdir(dest_loc2);

files=glob.glob(data_path+'*.wav');
for split_files in np.arange(0,len(files)):
    sig, rate=librosa.load(data_loc+j+'/Eko/'+k,sr=None)
    x = augment.EffectChain().reverb(50, 50, 50).channels(1).apply(torch.from_numpy(sig), src_info={'rate': rate})
    x=x.numpy().squeeze();    
    sf.write(dest_loc1+files[split_files][28:],x,rate)

    noise_generator = lambda: torch.zeros_like(torch.from_numpy(sig)).uniform_()
    snr=random.sample(range(5, 40), 1)
    y = augment.EffectChain().additive_noise(noise_generator, snr=int(snr[0])).apply(torch.from_numpy(sig), src_info={'rate': rate})
    y=y.numpy().squeeze(); 
    sf.write(dest_loc2+files[split_files][28:],y,rate)

In [82]:
augment_type=['reverb','uniform_noise']
set_loc = "./Sets/"
data_loc = "./"
for i in os.listdir(set_loc):
    set_full = np.loadtxt(set_loc + i, dtype=str)
    for j in tqdm(set_full):
        os.makedirs('./Augmented_Data/'+'Aug'+j)
        os.makedirs('./Augmented_Data/'+'Aug'+j+'/Eko')
        os.makedirs('./Augmented_Data/'+'Aug'+j+'/Mic')
        for k in os.listdir(data_loc+j+'/Eko'):
            sig, rate=librosa.load(data_loc+j+'/Eko/'+k,sr=None)
            x = augment.EffectChain().reverb(50, 50, 50).channels(1).apply(torch.from_numpy(sig), src_info={'rate': rate})
            x=x.numpy().squeeze();    
            sf.write('./Augmented_Data/'+'Aug'+j+'/Eko/reverb'+k,x,rate)
            
            noise_generator = lambda: torch.zeros_like(torch.from_numpy(sig)).uniform_()
            snr=random.sample(range(5, 40), 1)
            y = augment.EffectChain().additive_noise(noise_generator, snr=int(snr[0])).apply(torch.from_numpy(sig), src_info={'rate': rate})
            y=y.numpy().squeeze(); 
            sf.write('./Augmented_Data/'+'Aug'+j+'/Eko/uniform_noise'+k,y,rate)
            
            z = x+y 
            sf.write('./Augmented_Data/'+'Aug'+j+'/Eko/both'+k,z,rate)
            
            
            
            
        for k in os.listdir(data_loc+j+'/Mic'):
            sig, rate=librosa.load(data_loc+j+'/Mic/'+k,sr=16000)
            x = augment.EffectChain().reverb(50, 50, 50).channels(1).apply(torch.from_numpy(sig), src_info={'rate': rate})
            x=x.numpy().squeeze();    
            sf.write('./Augmented_Data/'+'Aug'+j+'/Mic/reverb'+k,x,rate)

            noise_generator = lambda: torch.zeros_like(torch.from_numpy(sig)).uniform_()
            snr=random.sample(range(5, 40), 1)
            y = augment.EffectChain().additive_noise(noise_generator, snr=int(snr[0])).apply(torch.from_numpy(sig), src_info={'rate': rate})
            y=y.numpy().squeeze(); 
            sf.write('./Augmented_Data/'+'Aug'+j+'/Mic/uniform_noise'+k,y,rate)

            z = x+y 
            sf.write('./Augmented_Data/'+'Aug'+j+'/Mic/both'+k,z,rate)



  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [00:24<02:26, 24.35s/it]

 29%|██▊       | 2/7 [00:44<01:55, 23.06s/it]

 43%|████▎     | 3/7 [01:10<01:36, 24.02s/it]

 57%|█████▋    | 4/7 [01:37<01:14, 24.86s/it]

 71%|███████▏  | 5/7 [02:00<00:48, 24.37s/it]

 86%|████████▌ | 6/7 [02:29<00:25, 25.68s/it]

100%|██████████| 7/7 [02:58<00:00, 26.74s/it]

  0%|          | 0/8 [00:00<?, ?it/s]

 12%|█▎        | 1/8 [00:30<03:33, 30.53s/it]

 25%|██▌       | 2/8 [00:56<02:54, 29.08s/it]

 38%|███▊      | 3/8 [01:25<02:26, 29.24s/it]

 50%|█████     | 4/8 [01:48<01:49, 27.30s/it]

 62%|██████▎   | 5/8 [02:16<01:22, 27.36s/it]

 75%|███████▌  | 6/8 [02:46<00:56, 28.19s/it]

 88%|████████▊ | 7/8 [03:12<00:27, 27.64s/it]

100%|██████████| 8/8 [03:43<00:00, 28.68s/it]

  0%|          | 0/7 [00:00<?, ?it/s]

 14%|█▍        | 1/7 [00:27<02:42, 27.15s/it]

 29%|██▊       | 2/7 [00:55<02:17, 27.52s/it]

 43%|████▎     | 3/7 [01:22<01:49, 27.39s/it]

 57%|█████▋    | 4/7 [01:49<01:22, 

In [77]:
sig, rate=librosa.load(data_loc+'2019_11_08_Vaishnavi_F_23_153_48_Y_Asthma_post/Mic/Vaishnavi_Mic_2019_11_08_13_25_38.990_B_R_T_DPB_1.wav',sr=16000)

In [91]:
snr=random.sample(range(5, 40), 1)

In [98]:
np.max(sig)

0.12964928

In [86]:
sig.reshape(sig.shape[0],1).shape

(581673, 1)

In [40]:
os.makedirs(data_loc+j+'/'+'Augment_Eko')

In [39]:
data_loc+j+'/'+'Augment_Eko'

'./2019_10_11_Bharathi_F_36_162_75_N_HC_NA/Augment_Eko'